### Importing the libraries and CSMAR data

In [1]:
import pandas as pd
import numpy as np

In [2]:
STdata = pd.read_excel(r'C:\Users\Sean\Documents\File Dump\HSBC PKU\STUDIES\Fourth Module\Investments II\Assignments\Assignment 4\Stock Trading data.xlsx')
SZESSEStockdata = pd.read_excel(r'C:\Users\Sean\Documents\File Dump\HSBC PKU\STUDIES\Fourth Module\Investments II\Assignments\Assignment 4\SZE and SSE stock codes.xls')

In [3]:
STdata.shape

(611754, 17)

### Preliminary data cleaning

Filtering in SSE/ SZE A-share stocks

In [4]:
STdata = STdata.merge(SZESSEStockdata, how='inner', on=['Stock Code'])

Renaming columns

In [5]:
STdata = STdata.rename(columns={"Date_x": "Date",})

In [6]:
STdata.shape

(584960, 25)

In [7]:
STdata.columns

Index(['Stock Code', 'Date', 'Opening Month Day', 'Opening Price',
       'Closing Month Day', 'Closing Price',
       'Number of Shares Traded in Month', 'Value of Shares Traded in Month',
       'Market Value of Tradable Shares', 'Total Market Value',
       'Number of Days of Monthly Trading',
       'Monthly Return With Cash Dividend Reinvested',
       'Monthly Return Without Cash Dividend Reinvested', 'Market Type',
       'Share Outstanding Change Date', 'Monhtly After-hours Trading Volume',
       'Monthly After-hours Trading Turnover', 'Date_y', 'SCode', 'Name',
       'Pre-close', 'Close', 'Change(%)', 'Trading Value(10,000)',
       'P/E Ratio'],
      dtype='object')

A look at the CSMAR data

In [8]:
STdata

,Stock Code,Date,Opening Month Day,Opening Price,Closing Month Day,Closing Price,Number of Shares Traded in Month,Value of Shares Traded in Month,Market Value of Tradable Shares,Total Market Value,...,Monhtly After-hours Trading Volume,Monthly After-hours Trading Turnover,Date_y,SCode,Name,Pre-close,Close,Change(%),"Trading Value(10,000)",P/E Ratio
0,1,1991-04,3,49.000,30,43.680,13400,615000.0,1157520.00,2118487.47,...,NaN,NaN,2021-06-16,000001.SZ,PAB,23.22,23.26,0.17,"87,847.08",16.61
1,1,1991-05,2,43.460,31,38.340,187800,7675000.0,1016010.00,1859496.56,...,NaN,NaN,2021-06-16,000001.SZ,PAB,23.22,23.26,0.17,"87,847.08",16.61
2,1,1991-06,1,38.530,28,33.990,30600,1094000.0,900735.00,1648520.81,...,NaN,NaN,2021-06-16,000001.SZ,PAB,23.22,23.26,0.17,"87,847.08",16.61
3,1,1991-07,1,33.650,31,29.540,6100,194043.0,782810.00,1432695.05,...,NaN,NaN,2021-06-16,000001.SZ,PAB,23.22,23.26,0.17,"87,847.08",16.61
4,1,1991-08,1,29.390,31,15.000,3243100,49576242.0,674833.82,1346274.65,...,NaN,NaN,2021-06-16,000001.SZ,PAB,23.22,23.26,0.17,"87,847.08",16.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584955,900957,2020-12,1,0.520,31,0.503,3700329,1863830.0,92552.00,175547.00,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
584956,900957,2021-01,4,0.515,29,0.671,34446674,22584670.0,123464.00,234179.00,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
584957,900957,2021-02,1,0.667,26,0.732,16693107,11753255.0,134688.00,255468.00,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
584958,900957,2021-03,1,0.725,31,0.691,7539322,5267485.0,127144.00,241159.00,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data wrangling
Variables to be created: SIZE, DVOL, TURN, PRICE

In [9]:
DVOL_nonlog = STdata.groupby("Stock Code")['Value of Shares Traded in Month'].shift(1)
TURN_nonlog = STdata.groupby("Stock Code")["Value of Shares Traded in Month"].shift(1)/STdata.groupby("Stock Code")["Market Value of Tradable Shares"].shift(1)
DVOL = DVOL_nonlog.apply(np.log)
TURN = TURN_nonlog.apply(np.log)
STdata["DVOL-nl"] = DVOL_nonlog
STdata["TURN-nl"] = TURN_nonlog

In [10]:
RegTable1 = STdata["Stock Code"].to_frame()
RegTable1["Date"] = STdata["Date"]
RegTable1["SIZE"] = STdata.groupby("Stock Code")['Total Market Value'].shift(1).apply(np.log)
RegTable1["DVOL"] = DVOL
RegTable1["TURN"] = TURN
RegTable1["PRICE"] = 1/(STdata.groupby("Stock Code")["Closing Price"].shift(1)).apply(np.log)

Variables to be created: STDVOL, CVVOL, STDTURN, CVTURN, RET2-3, RET4-6, RET7-12

In [11]:
RegTable1.insert(6,"STDVOL",STdata['DVOL-nl'].rolling(36,1).std().shift().apply(np.log),True)
RegTable1.insert(6,"STDTURN",STdata['TURN-nl'].rolling(36,1).std().shift().apply(np.log),True)
RegTable1.insert(6,"CVVOL",(STdata['DVOL-nl'].rolling(36,1).std().shift().apply(np.log))/(STdata['DVOL-nl'].rolling(36,1).mean().shift().apply(np.log)),True)
RegTable1.insert(6,"CVTURN",(STdata['TURN-nl'].rolling(36,1).std().shift().apply(np.log))/(STdata['TURN-nl'].rolling(36,1).mean().shift().apply(np.log)),True)
STdata["Monthly Return"] = STdata["Monthly Return Without Cash Dividend Reinvested"]+1
RegTable1.insert(6,"RET2-3",STdata['Monthly Return'].rolling(2,1).apply(np.prod).shift(),True)
RegTable1.insert(6,"RET4-6",STdata['Monthly Return'].rolling(3,3).apply(np.prod).shift(),True)
RegTable1.insert(6,"RET7-12",STdata['Monthly Return'].rolling(6,6).apply(np.prod).shift(),True)

C:\Users\Sean\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  
C:\Users\Sean\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys
C:\Users\Sean\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' 

Adding the variable for monthly return and excluding stocks with less than 36 months

In [12]:
RT=RegTable1
RT["Monthly Return"] = STdata["Monthly Return Without Cash Dividend Reinvested"]
RT = RegTable1.groupby('Stock Code', group_keys=False).apply(lambda x:x.iloc[36:])

In [13]:
RT = RT[RT.duplicated('Stock Code', keep=False)]
RT = RT[RT.groupby('Stock Code')['Stock Code'].transform('size') > 36]

Reading the CSMAR file on market monthly returns

In [14]:
MRdata = pd.read_excel(r'C:\Users\Sean\Documents\File Dump\HSBC PKU\STUDIES\Fourth Module\Investments II\Assignments\Assignment 4\data\TRD_Mont.xlsx')

In [15]:
MRdata = MRdata.rename(columns={"Trdmnt": "Date",})

Appending SSE and SZE market monthly returns by stock and calculating excess return

In [16]:
SZEMRdata = MRdata.drop(MRdata[MRdata['Markettype'] != 1].index)
SSEMRdata = MRdata.drop(MRdata[MRdata['Markettype'] != 4].index)
SZEVTdata = RT.drop(RT[RT['Stock Code'] > 310000].index)
SSEVTdata = RT.drop(RT[RT['Stock Code'] < 310000].index)
DD = SZEMRdata[["Date","Mretmdtl"]].copy()
FF = SSEMRdata[["Date","Mretmdtl"]].copy()

In [17]:
SZEVTdata=SZEVTdata.merge(DD, on='Date', how='left')
SSEVTdata=SSEVTdata.merge(FF, on='Date', how='left')
SSEVTdata["Excess Return"] = SSEVTdata["Monthly Return"]-SSEVTdata["Mretmdtl"]
SZEVTdata["Excess Return"] = SZEVTdata["Monthly Return"]-SZEVTdata["Mretmdtl"]

In [18]:
VariablesTable = SZEVTdata.append(SSEVTdata, ignore_index=True).drop(['Monthly Return', 'Mretmdtl'], axis=1)

### Final table

In [19]:
VariablesTable

,Stock Code,Date,SIZE,DVOL,TURN,PRICE,RET7-12,RET4-6,RET2-3,CVTURN,CVVOL,STDTURN,STDVOL,Excess Return
0,1,1994-04,15.215397,18.876623,4.279802,0.368816,0.665929,0.673378,0.833795,0.921899,0.992582,4.699679,19.819621,0.227418
1,1,1994-05,15.275983,19.736639,5.079232,0.360755,0.731807,0.885872,0.880991,0.923812,0.993203,4.694984,19.813538,-0.014116
2,1,1994-06,15.185041,20.614841,6.048376,0.372992,0.657657,0.804408,0.970100,0.912492,0.991165,4.662303,19.794855,0.079296
3,1,1994-07,15.102254,18.799468,4.105409,0.384877,0.601342,0.893023,0.840526,0.909244,0.988380,4.707026,19.789729,0.264968
4,1,1994-08,15.053464,18.850581,4.205312,0.480898,0.710413,0.801936,0.878285,0.903607,0.987407,4.686230,19.778221,-0.885891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434724,900957,2020-12,12.112740,14.833841,3.361238,-1.538254,1.335039,0.957798,1.019531,0.925732,0.978013,3.473792,15.063796,-0.058306
434725,900957,2021-01,12.075662,14.438144,3.002618,-1.455254,0.976699,0.982422,0.976699,0.929996,0.978999,3.474369,15.039601,0.332236
434726,900957,2021-02,12.363841,16.932782,5.209077,-2.506353,1.158895,1.302913,1.285441,0.930288,0.980277,3.474901,15.048212,0.091042
434727,900957,2021-03,12.450852,16.279641,4.468925,-3.205387,1.343120,1.402299,1.455268,0.965764,0.992939,3.687041,15.322215,-0.026359


### Saving as excel and csv files

In [20]:
# writing to Excel
datatoexcel3 = pd.ExcelWriter('FinalVarTable2.xlsx')
  
# write DataFrame to excel
VariablesTable.to_excel(datatoexcel3)
  
# save the excel
datatoexcel3.save()
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [21]:
VariablesTable.to_csv(r'C:\Users\Sean\Documents\File Dump\HSBC PKU\STUDIES\Fourth Module\Investments II\Assignments\Assignment 4\FinalVarTable.csv', index = False)